In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
base_url = 'https://www.themoviedb.org/movie'
temp_url = '/top-rated?page='
all_page_data = []

for num in range(1, 2):  # pages
    resp1 = requests.get(base_url + temp_url + str(num)).text
    soup_data = BeautifulSoup(resp1, 'lxml')
    all_div = soup_data.find_all('div', class_='card style_1')

    for items in all_div:
        inner_div = items.find('div', class_='content')
        inner_link = inner_div.find('a')['href']
        full_link = 'https://www.themoviedb.org' + inner_link

        inner_data_req = requests.get(full_link).text
        new_soup_data = BeautifulSoup(inner_data_req, 'lxml')

        movie_name = inner_div.find('h2').text.strip()
        movie_date = inner_div.find('p').text.strip()

        rating_div = new_soup_data.find('div', 'user_score_chart')
        if rating_div:
            rating = rating_div["data-percent"]
        else:
            rating = 'N/A'

        genre_class = new_soup_data.find('span', class_='genres')
        genre = genre_class.findAll('a')
        genre_list = [i.text for i in genre]  # Store as a list

        run_time_find = new_soup_data.find('span', class_='runtime')
        if run_time_find:
            run_time = run_time_find.text.strip()
        else:
            run_time = "N/A"

        # Extract certification
        certification_span = new_soup_data.find('span', class_='certification')
        certification = certification_span.get_text(strip=True) if certification_span else 'N/A'

        # Extract overview
        ovr_view = new_soup_data.find('div', class_='overview')
        overview = ovr_view.find('p').text

        # Extract tagline
        tagline_h3 = new_soup_data.find('h3', class_='tagline')
        tagline = tagline_h3.get_text(strip=True) if tagline_h3 else 'N/A'

        # Locate the facts section directly
        facts_section = new_soup_data.find('section', class_='facts left_column')

        # Initialize variables for language, budget, and revenue
        language = 'N/A'
        budget = 'N/A'
        revenue = 'N/A'

        # Extract language, budget, and revenue
        if facts_section:
            # Extract Original Language
            language_bdi = facts_section.find('bdi', string='Original Language')
            if language_bdi:
                language_p = language_bdi.find_parent('p')
                if language_p:
                    language = language_p.get_text(strip=True).replace('Original Language', '').strip()

            # Extract Budget
            budget_bdi = facts_section.find('bdi', string='Budget')
            if budget_bdi:
                budget_p = budget_bdi.find_parent('p')
                if budget_p:
                    budget = budget_p.get_text(strip=True).replace('Budget', '').strip()

            # Extract Revenue
            revenue_bdi = facts_section.find('bdi', string='Revenue')
            if revenue_bdi:
                revenue_p = revenue_bdi.find_parent('p')
                if revenue_p:
                    revenue = revenue_p.get_text(strip=True).replace('Revenue', '').strip()

        # Extract Directors
        ol_profile = new_soup_data.find('ol', class_='people no_image')
        li_profile = ol_profile.findAll('li', class_='profile')
        director = set()
        for i in range(0, len(li_profile)):
            if 'Director' in li_profile[i].find('p', class_="character").text:
                direct = li_profile[i].find('a').text
                director.add(direct)
        directors = list(director)  # Convert set to list

        # Extract Cast
        cast_list = []
        cast_section = new_soup_data.find('ol', class_='people scroller')
        if cast_section:
            cast_cards = cast_section.find_all('li', class_='card')
            for card in cast_cards:
                actor_p = card.find('p')
                if actor_p:
                    actor_a = actor_p.find('a')
                    if actor_a:
                        cast_list.append(actor_a.get_text(strip=True))

        # Extract keywords
        keyword_rounded = []
        keyword_bold = []
        keywords_section = new_soup_data.find('section', class_='keywords right_column')
        if keywords_section:
            ul_tag = keywords_section.find('ul')
            if ul_tag:
                for li in ul_tag.find_all('li'):
                    a_tag = li.find('a')
                    if a_tag:
                        if 'rounded' in a_tag.get('class', []):
                            keyword_rounded.append(a_tag.get_text(strip=True))
                        elif '!border' in a_tag.get('class', []):
                            keyword_bold.append(a_tag.get_text(strip=True))



        # Extract Reviews
        reviews_list = []
        inner_content_div = new_soup_data.find('div', class_='inner_content')
        if inner_content_div:
            reviews_link = inner_content_div.find('p', class_='new_button')
            if reviews_link:
                reviews_url = 'https://www.themoviedb.org' + reviews_link.find('a')['href']
                reviews_page = requests.get(reviews_url).text
                reviews_soup = BeautifulSoup(reviews_page, 'lxml')

                review_containers = reviews_soup.find_all('div', class_='review_container')
                for container in review_containers:
                    review_contents = container.find_all('div', class_='content')
                    for review_content in review_contents:
                        writer_h5 = review_content.find('h5')
                        writer = writer_h5.find('a').text if writer_h5 and writer_h5.find('a') else 'N/A'

                        # Modified code for extracting teaser and review text
                        teaser_div = review_content.find('div', class_='teaser')
                        if teaser_div:
                            read_more_link = teaser_div.find('a', class_='underline')
                            if read_more_link:
                                full_review_url = 'https://www.themoviedb.org' + read_more_link['href']
                                full_review_page = requests.get(full_review_url).text
                                full_review_soup = BeautifulSoup(full_review_page, 'lxml')
                                full_review_div = full_review_soup.find('div', class_='content column pad')
                                if full_review_div:
                                    # Find all <p> tags inside the full review div
                                    all_paragraphs = full_review_div.find_all('p')
                                    # Join the text from all <p> tags
                                    review_text = ' '.join(p.get_text(strip=True) for p in all_paragraphs)
                                else:
                                    review_text = teaser_div.get_text(strip=True)
                            else:
                                review_text = teaser_div.get_text(strip=True)

                            reviews_list.append({
                                'Writer': writer,
                                'Review': review_text
                            })



        # Add all data to the dictionary
        first_page_data = {
            'Movie_name': movie_name,
            'Release_date': movie_date,
            'Rating': rating,
            'Genre': genre_list, 
            'Run_time': run_time,
            'Certification': certification,
            'Overview': overview,
            'Tagline': tagline,
            'Director': directors,  
            'Language': language,
            'Budget': budget,
            'Revenue': revenue,
            'Keyword_Rounded': keyword_rounded,
            'Keyword_Bold': keyword_bold,  
            'Cast': cast_list,
            'Reviews':  reviews_list,
        }

        all_page_data.append(first_page_data)


/var/folders/nv/nf693pxs72lbcg7sm8bhb3dw0000gn/T/ipykernel_57788/1666819223.py:28: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  genre = genre_class.findAll('a')
/var/folders/nv/nf693pxs72lbcg7sm8bhb3dw0000gn/T/ipykernel_57788/1666819223.py:82: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  li_profile = ol_profile.findAll('li', class_='profile')


In [6]:
df = pd.DataFrame(all_page_data)
df.head()

,Movie_name,Release_date,Rating,Genre,Run_time,Certification,Overview,Tagline,Director,Language,Budget,Revenue,Keyword_Rounded,Keyword_Bold,Cast,Reviews
0,The Shawshank Redemption,17 Feb 1995,87,"[Drama, Crime]",2h 22m,15,Imprisoned in the 1940s for the double murder ...,Fear can hold you prisoner. Hope can set you f...,[Frank Darabont],English,"$25,000,000.00","$28,341,469.00","[prison, corruption, police brutality, based o...",[admiring],"[Morgan Freeman, Tim Robbins, Bob Gunton, Will...","[{'Writer': 'elshaarawy', 'Review': 'very good..."
1,Selena Gomez: My Mind & Me,04 Nov 2022,87,"[Documentary, Music]",1h 35m,12,"After years in the limelight, Selena Gomez ach...","Every breath, a breakthrough.",[Alek Keshishian],English,-,-,[music documentary],[],"[Selena Gomez, Raquelle Stevens, Ashley Cook]",[]
2,The Godfather,24 Aug 1972,87,"[Drama, Crime]",2h 55m,15,"Spanning the years 1945 to 1955, a chronicle o...",An offer you can't refuse.,[Francis Ford Coppola],English,"$6,000,000.00","$245,066,411.00","[italy, loss of loved one, love at first sight...","[macabre, aggressive, vindictive, suspenseful,...","[Marlon Brando, Al Pacino, James Caan, Robert ...","[{'Writer': 'futuretv', 'Review': 'The Godfath..."
3,The Godfather Part II,20 Dec 1974,86,"[Drama, Crime]",3h 22m,15,In the continuing saga of the Corleone crime f...,The rise and fall of the Corleone empire.,[Francis Ford Coppola],English,"$13,000,000.00","$102,600,000.00","[italy, new york city, new year's eve, italian...","[melancholy, aggressive, vindictive, cautionar...","[Al Pacino, Robert Duvall, Diane Keaton, Rober...","[{'Writer': 'jkbbr549', 'Review': 'This is by ..."
4,Schindler's List,18 Feb 1994,86,"[Drama, History, War]",3h 15m,15,The true story of how businessman Oskar Schind...,"Whoever saves one life, saves the world entire.",[Steven Spielberg],English,"$22,000,000.00","$321,365,567.00","[factory, concentration camp, hero, holocaust ...","[disturbed, philosophical, loving, hopeful]","[Liam Neeson, Ben Kingsley, Ralph Fiennes, Car...","[{'Writer': 'Mayurpanchamia', 'Review': 'Direc..."


In [7]:
df['Reviews'][0]

[{'Writer': 'elshaarawy', 'Review': 'very good movie 9.5/10 محمد الشعراوى'},
 {'Writer': 'John Chard',
  'Review': 'Some birds aren\'t meant to be caged. The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them. There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn\'t give much inkling to anyone about what it was

In [8]:
df.index = df.index+1
df.to_csv('all_movies_datas.csv',index=True)  